In [ ]:
#pip install -q -U google-genai

# Importacion de librerias

In [1]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

# Librerias basicas
import os, tempfile, glob, random, json
from pathlib import Path
import numpy as np
from itertools import combinations
from uuid import uuid4
from PIL import Image
import chromadb

from IPython.display import Markdown
from getpass import getpass

# Para el entorno de google
from google import genai
from google.genai import types

# Langchain 
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.runnables import RunnableLambda, RunnableParallel, RunnablePassthrough, RunnableSequence
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string, ChatMessage
from langchain_core.documents import Document
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_transformers import EmbeddingsRedundantFilter,LongContextReorder
from langchain_community.chat_message_histories import StreamlitChatMessageHistory
from operator import itemgetter
from langchain_core.prompts.base import format_document


# Document Loaders
from langchain_community.document_loaders import (
    PyPDFLoader,
    TextLoader,
    DirectoryLoader,
    CSVLoader,
    UnstructuredExcelLoader,
    Docx2txtLoader,
    JSONLoader
)

# Text Splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter

# Chroma: vectorstore
from langchain_chroma import Chroma


## Api key de google gemini

In [ ]:
# Entorno de Langchain
os.environ["GOOGLE_API_KEY"] = ""

In [ ]:
# Entorno de google / genai
gemini_client = genai.Client(api_key="")

In [ ]:
# test embedding
result = gemini_client.models.embed_content(
        model="gemini-embedding-001",
        contents="What is the meaning of life?")

print(result.embeddings)

[ContentEmbedding(
  values=[
    -0.022374554,
    -0.004560777,
    0.013309286,
    -0.0545072,
    -0.02090443,
    <... 3067 more items ...>,
  ]
)]


In [ ]:
print(len(result.embeddings[0].values))

3072


# Cargado de datos

## Cargado del JSON

In [ ]:
with open("../resources/cpe.json", "r", encoding="utf-8") as f:
    data = json.load(f)

docs = []

In [ ]:
for item in data:
    tipo = item.get("tipo", "").capitalize()
    text = ""

    if tipo == "Introduccion":
        text = f"Título: {item.get('titulo', '')}\n" \
                f"Subtítulo: {item.get('subtitulo', '')}\n" \
                f"{item.get('contenido', '')}"

    elif tipo == "Articulo":
        text = (
            f"Parte {item.get('parte_num', '')}: {item.get('parte_nom', '')}\n"
            f"Título {item.get('titulo_num', '')}: {item.get('titulo_nom', '')}\n"
            f"Capítulo {item.get('capitulo_num', '')}: {item.get('capitulo_nom', '')}\n"
            f"Sección {item.get('seccion_num', '')}: {item.get('seccion_nom', '')}\n"
            f"Artículo {item.get('art_num', '')}: {item.get('nombre_juridico', '')}\n"
            f"{item.get('contenido', '')}"
        )
        
    elif tipo == "Disposición":
        text = (
            f"Disposición {item.get('disposicion', '')}\n"
            f"Nombre jurídico: {item.get('nombre_juridico', '')}\n"
            f"{item.get('contenido', '')}"
        )

    # Pongo este else, porque sino me sale error en page_content=text, porque es posible que text este vacio
    else:
        text = item.get("contenido", "")

    docs.append(Document(page_content=text, metadata={"tipo": tipo}))

print(f"Se cargaron {len(docs)} documentos desde el JSON.")

In [ ]:
# Solamente para testear que se hayan cargado los documentos correctamente, agarramos uno aleatorio y lo analizamos
import random
random_document_id = random.choice(range(len(docs)))

print("test: ", random_document_id)
print(docs[random_document_id])

## Asignamiento de IDs

In [ ]:
ids = []
metas = []
cont = 0

for i in range(len(docs)):
    if (i<2):
        ids.append(f"introduccion_{i+1}")
        metas.append({"tipo": "introduccion", "número": i+1})
    elif(i<399):
        ids.append(f"artículo_{i-1}")
        metas.append({"tipo": "artículo", "número": i-1})
    elif(i==399):
        ids.append(f"artículo_398_A")
        metas.append({"tipo": "artículo", "número": "398_A"})
    elif(i==400):
        ids.append(f"artículo_398_B")
        metas.append({"tipo": "artículo", "número": "398_B"})
    elif(i<414):
        ids.append(f"artículo_{i-2}")
        metas.append({"tipo": "artículo", "número": i-2})
    else:
        cont += 1
        ids.append(f"disposición_{cont}")
        metas.append({"tipo": "disposicion", "número": cont})

assert len(docs) == len(ids) == len(metas)

# Embedding part

### Gemini embedding class

In [22]:
class GeminiEmbeddingFunction:
    def __init__(self, model="gemini-embedding-001"):
        self.model = model

    def __call__(self, input):
        result = gemini_client.models.embed_content(
            model=self.model,
            contents=input,
            config=types.EmbedContentConfig(task_type="retrieval_document")
        )
        return [emb.values for emb in result.embeddings]
    
    def embed_query(self, input):
        result = gemini_client.models.embed_content(
            model=self.model,
            contents=input,
            config=types.EmbedContentConfig(task_type="retrieval_query")
        )
        return [emb.values for emb in result.embeddings]

### Create chroma db

In [ ]:
# Funcion para crear la base de datos vectorial (Caso de no tener limite de batch)
def create_chroma_db(documents, ids, name):
    chroma_client = chromadb.Client()
    db = chroma_client.create_collection(
        name=name,
        embedding_function=GeminiEmbeddingFunction()
    )
    db.add(
        documents=[d.page_content for d in documents],
        ids=ids
    )
    return db

In [28]:
# Funcion para crear la base de datos vectorial con batch limitado (no es limite de API)
def create_chroma_db(documents, ids, name, batch_size=100):
    import time
    chroma_client = chromadb.PersistentClient(path="./vector/" + name)

    db = chroma_client.create_collection(
        name=name,
        embedding_function=GeminiEmbeddingFunction()
    )

    print(f"Insertando {len(documents)} documentos en lotes de {batch_size}...")

    for i in range(0, len(documents), batch_size):
        batch_docs = documents[i:i+batch_size]
        batch_ids = ids[i:i+batch_size]

        db.add(
            documents=[d.page_content for d in batch_docs],
            ids=batch_ids
        )

        print(f"Lote {i//batch_size + 1} insertado ({len(batch_docs)} docs)")
        time.sleep(5)

    print("Base de datos creada exitosamente")
    return db

In [ ]:
db = create_chroma_db(docs, ids, "cpep_gemini_test5")

In [ ]:
import pandas as pd
sample_data = db.get(include=['documents', 'embeddings'])

df = pd.DataFrame({
    "IDs": sample_data['ids'][:15],
    "Documents": sample_data['documents'][:15],
    "Embeddings": [str(emb)[:150] + "..." for emb in sample_data['embeddings'][:15]]  # Truncate embeddings
})

df

### Cargar chroma db

In [ ]:
client = chromadb.PersistentClient(path="./vector/cpep_gemini_test5")

db = client.get_collection(name="cpep_gemini_test5")

db._embedding_function = GeminiEmbeddingFunction()

print("Total de documentos:", db.count())

Total de documentos: 426


In [ ]:
query = "¿Qué dice el artículo 398 de la constitución política del estado boliviano?"
results = db.query(query_texts=[query], n_results=5)

for i, doc in enumerate(results["documents"][0]):
    print(f"Documento {i+1}")
    print(doc[:400])

Documento 1
Parte Cuarta: Estructura y organización económica del Estado
Título 2: Medio Ambiente, Recursos Naturales, Tierra y Territorio
Capítulo Noveno: Tierra y territorio
Sección None: None
Artículo 398: opción A para el Referendo Dirimitorio (Prohibición del Latifundio y la Doble Titulación)
Se prohíbe el latifundio y la doble titulación por ser contrarios al interés colectivo y al desarrollo del país. 
Documento 2
Parte Cuarta: Estructura y organización económica del Estado
Título 2: Medio Ambiente, Recursos Naturales, Tierra y Territorio
Capítulo Noveno: Tierra y territorio
Sección None: None
Artículo 398: opción B para el Referendo Dirimitorio (Prohibición del Latifundio y la Doble Titulación)
Se prohíbe el latifundio y la doble titulación por ser contrarios al interés colectivo y al desarrollo del país. 
Documento 3
Parte Primera: Bases fundamentales del estado derechos, deberes y garantías
Título 2: Derechos Fundamentales y Garantías
Capítulo Sexto: Educación, interculturali

In [ ]:
print("vector_store:", db.count(), "documentos.")
print(db.count())
print(db.peek())

vector_store_gemini: 426 documentos.
426
{'ids': ['introduccion_1', 'introduccion_2', 'artículo_1', 'artículo_2', 'artículo_3', 'artículo_4', 'artículo_5', 'artículo_6', 'artículo_7', 'artículo_8'], 'embeddings': array([[ 0.00245742,  0.04283391,  0.01778333, ...,  0.02235511,
        -0.00752448, -0.00406222],
       [-0.00149028,  0.01373982,  0.04177929, ...,  0.0159325 ,
         0.01358084,  0.00048896],
       [-0.00251971,  0.02007411,  0.03636278, ...,  0.01281254,
         0.00248115, -0.01281189],
       ...,
       [-0.01256879,  0.01323783,  0.02546843, ...,  0.02831057,
        -0.00088528, -0.02769176],
       [-0.01420504,  0.01602528,  0.02913002, ...,  0.00889516,
         0.00680231, -0.0045351 ],
       [-0.01655631,  0.00083472,  0.02253124, ...,  0.02003035,
         0.02303935,  0.00135051]], shape=(10, 3072)), 'documents': ['Título: Antecedentes Legales\nSubtítulo: None\nLa Constitución Política del Estado, promulgada el 7 de febrero de 2009, consta de 5 Partes: 

## Similarity Search

In [ ]:
from numpy import dot
from numpy.linalg import norm

def print_documents_distances(results, show_score=False):
    if show_score:
        for i, (doc, score) in enumerate(zip(results["documents"][0], results["distances"][0])):
            print(f"{'-'*80}")
            print(f"Documento {i+1} | score={round(score, 4)}")
            print(doc[:400])
            print("\n")
    else:
        for i, doc in enumerate(results["documents"][0]):
            print(f"{'-'*80}")
            print(f"Documento {i+1}")
            print(doc[:400])
            print("\n")

def print_documents_similarity(results, show_score=False):
    if show_score:
        for i, (doc, dist) in enumerate(zip(results["documents"][0], results["distances"][0])):
            similarity = 1 - dist  # convertir distancia a similitud
            print(f"{'-'*80}")
            print(f"Documento {i+1} | similitud={round(similarity,4)}")
            print(doc[:400])
            print("\n")
    else:
        for i, doc in enumerate(results["documents"][0]):
            print(f"{'-'*80}")
            print(f"Documento {i+1}")
            print(doc[:400])
            print("\n")


In [ ]:

query = "¿Qué dice el artículo 398 de la constitución política del estado boliviano?"
results = db.query(query_texts=[query], n_results=5)
print_documents_distances(results, show_score=True)


--------------------------------------------------------------------------------
📄 Documento 1 | score=0.4547
Parte Cuarta: Estructura y organización económica del Estado
Título 2: Medio Ambiente, Recursos Naturales, Tierra y Territorio
Capítulo Noveno: Tierra y territorio
Sección None: None
Artículo 398: opción A para el Referendo Dirimitorio (Prohibición del Latifundio y la Doble Titulación)
Se prohíbe el latifundio y la doble titulación por ser contrarios al interés colectivo y al desarrollo del país. 

--------------------------------------------------------------------------------
📄 Documento 2 | score=0.4579
Parte Cuarta: Estructura y organización económica del Estado
Título 2: Medio Ambiente, Recursos Naturales, Tierra y Territorio
Capítulo Noveno: Tierra y territorio
Sección None: None
Artículo 398: opción B para el Referendo Dirimitorio (Prohibición del Latifundio y la Doble Titulación)
Se prohíbe el latifundio y la doble titulación por ser contrarios al interés colectivo y a

In [ ]:
print_documents_similarity(results, show_score=True)


--------------------------------------------------------------------------------
📄 Documento 1 | similitud=0.5453
Parte Cuarta: Estructura y organización económica del Estado
Título 2: Medio Ambiente, Recursos Naturales, Tierra y Territorio
Capítulo Noveno: Tierra y territorio
Sección None: None
Artículo 398: opción A para el Referendo Dirimitorio (Prohibición del Latifundio y la Doble Titulación)
Se prohíbe el latifundio y la doble titulación por ser contrarios al interés colectivo y al desarrollo del país. 

--------------------------------------------------------------------------------
📄 Documento 2 | similitud=0.5421
Parte Cuarta: Estructura y organización económica del Estado
Título 2: Medio Ambiente, Recursos Naturales, Tierra y Territorio
Capítulo Noveno: Tierra y territorio
Sección None: None
Artículo 398: opción B para el Referendo Dirimitorio (Prohibición del Latifundio y la Doble Titulación)
Se prohíbe el latifundio y la doble titulación por ser contrarios al interés colec

In [ ]:
embedding_adapter = GeminiEmbeddingFunction()
query_embeddings = embedding_adapter.embed_query(query)[0]

docs_texts = results["documents"][0]

docs_embeddings = [embedding_adapter(text)[0] for text in docs_texts]

for i, doc_emb in enumerate(docs_embeddings):
    similarity = np.dot(query_embeddings, doc_emb) / (
        np.linalg.norm(query_embeddings) * np.linalg.norm(doc_emb)
    )
    print(f"Similaridad de documento_{i+1} con el query: {round(similarity,4)}")

🔹 Similaridad de documento_1 con el query: 0.7726
🔹 Similaridad de documento_2 con el query: 0.7711
🔹 Similaridad de documento_3 con el query: 0.7562
🔹 Similaridad de documento_4 con el query: 0.7542
🔹 Similaridad de documento_5 con el query: 0.7531


# Retrievers

### Vector Store-backed retriever

In [ ]:
import numpy as np

class ChromaGeminiRetriever:
    def __init__(self, chroma_collection, embedding_function, k=4, score_threshold=None):
        self.collection = chroma_collection
        self.embedding_function = embedding_function
        self.k = k
        self.score_threshold = score_threshold

    def retrieve(self, query):
        results = self.collection.query(query_texts=[query], n_results=self.k)

        if self.score_threshold is not None and "distances" in results:
            filtered_docs = []
            filtered_scores = []
            for doc, score in zip(results["documents"][0], results["distances"][0]):
                if score <= self.score_threshold:
                    filtered_docs.append(doc)
                    filtered_scores.append(score)
            results["documents"][0] = filtered_docs
            results["distances"][0] = filtered_scores

        return results

    def invoke(self, query):
        return self.retrieve(query)

In [ ]:
embedding_function = GeminiEmbeddingFunction()
retriever_gemini = ChromaGeminiRetriever(
    chroma_collection=db,
    embedding_function=embedding_function,
    k=5,                      
    score_threshold=None     
)

In [ ]:
query = "¿Qué dice el artículo 398 de la constitución política del estado boliviano?"
relevant_docs = retriever_gemini.invoke(query)
print_documents_similarity(relevant_docs, show_score=True)


--------------------------------------------------------------------------------
📄 Documento 1 | similitud=0.5453
Parte Cuarta: Estructura y organización económica del Estado
Título 2: Medio Ambiente, Recursos Naturales, Tierra y Territorio
Capítulo Noveno: Tierra y territorio
Sección None: None
Artículo 398: opción A para el Referendo Dirimitorio (Prohibición del Latifundio y la Doble Titulación)
Se prohíbe el latifundio y la doble titulación por ser contrarios al interés colectivo y al desarrollo del país. 

--------------------------------------------------------------------------------
📄 Documento 2 | similitud=0.5421
Parte Cuarta: Estructura y organización económica del Estado
Título 2: Medio Ambiente, Recursos Naturales, Tierra y Territorio
Capítulo Noveno: Tierra y territorio
Sección None: None
Artículo 398: opción B para el Referendo Dirimitorio (Prohibición del Latifundio y la Doble Titulación)
Se prohíbe el latifundio y la doble titulación por ser contrarios al interés colec

### ChatModel

In [ ]:
class ChatGemini:
    def __init__(self, model="gemini-2.5-flash-lite", temperature=0.3, memory=None):
        self.model = model
        self.temperature = temperature
        self.memory = memory

    def invoke(self, prompt):
        # Incluir memoria
        if self.memory and self.memory.history:
            context = self.memory.get_context()
            prompt = f"{context}\n\nUsuario: {prompt}\nAsistente:"
        
        response = gemini_client.models.generate_content(
            model=self.model,
            contents=prompt,
            config={
                "temperature": self.temperature,
                "max_output_tokens": 1024
            }
        )

        if self.memory:
            self.memory.add_message("user", prompt)
            self.memory.add_message("assistant", response.text)

        return response.text

In [45]:
# test
prompt_test = "¿Qué dice el artículo 398 de la Constitución Política del estado boliviano?"
llm = ChatGemini()
response = llm.invoke(prompt_test)

In [46]:
print(response)

El artículo 398 de la Constitución Política del Estado Plurinacional de Bolivia se refiere a la **propiedad de los recursos naturales y la participación de las comunidades indígenas y campesinas en su aprovechamiento**.

Aquí te presento el contenido del artículo:

**Artículo 398.**
"Los recursos naturales son de carácter estratégico y de interés soberano de las bolivianas y los bolivianos. El Estado, a través de las entidades y empresas públicas, se encargará de su exploración, explotación, industrialización, distribución y comercialización, en los términos que establezca la ley.

Las comunidades indígenas y campesinas, como titulares de la tierra, participarán en el aprovechamiento de los recursos naturales existentes en ella, en los términos que establezca la ley."

**En resumen, el artículo 398 establece dos puntos clave:**

1.  **Soberanía y Estrategia de los Recursos Naturales:** Los recursos naturales son considerados de gran importancia estratégica y pertenecen soberanamente a 

### Memory

In [ ]:
from datetime import datetime

class ConversationMemory:
    def __init__(self, max_turns=10):
        self.history = []
        self.max_turns = max_turns

    def add_message(self, role, content):
        # Agregar el mensaje al historial user o assistant
        self.history.append({
            "role": role,
            "content": content,
            "timestamp": datetime.now().isoformat()
        })

        if len(self.history) > self.max_turns * 2:
            self.history = self.history[-self.max_turns*2:]

    def get_context(self):
        if not self.history:
            return "No hay historial previo."
        return "\n".join([f"{m['role'].capitalize()}: {m['content']}" for m in self.history])

    def clear(self):
        self.history = []


In [ ]:
def reset_chat_session(llm):
    if llm.memory:
        llm.memory.clear()
        print(" Memoria temporal del chat borrada.")
    else:
        print("No hay memoria activa para borrar.")

In [ ]:
# Para resetear la memoria
reset_chat_session(llm)

### PromptTemplate

In [49]:
standalone_question_template = """
Dada la siguiente conversación y una pregunta de seguimiento,
reformula la pregunta de seguimiento para que sea una pregunta independiente,
manteniendo su significado original y en el mismo idioma (español).

Historial de chat:
{chat_history}

Pregunta de seguimiento:
{question}

Pregunta independiente:
"""

In [53]:
def build_reformulation_prompt(memory, question):
    chat_history = memory.get_context() if memory else "No hay historial previo."
    prompt = standalone_question_template.format(
        chat_history=chat_history,
        question=question
    )
    return prompt

In [54]:
#test 
memory = ConversationMemory(max_turns=5)

memory.add_message("user", "¿Qué dice el artículo 7 de la Constitución Política del Estado boliviano?")
memory.add_message("assistant", "El Artículo 7 establece los principios fundamentales del Estado Plurinacional de Bolivia.")

nueva_pregunta = "¿Y qué pasa si no se respetan esos derechos?"

prompt_reformulado = build_reformulation_prompt(memory, nueva_pregunta)
print(prompt_reformulado)


Dada la siguiente conversación y una pregunta de seguimiento,
reformula la pregunta de seguimiento para que sea una pregunta independiente,
manteniendo su significado original y en el mismo idioma (español).

Historial de chat:
User: ¿Qué dice el artículo 7 de la Constitución Política del Estado boliviano?
Assistant: El Artículo 7 establece los principios fundamentales del Estado Plurinacional de Bolivia.

Pregunta de seguimiento:
¿Y qué pasa si no se respetan esos derechos?

Pregunta independiente:



In [ ]:
respuesta_reformulada = llm.invoke(prompt_reformulado)
print(respuesta_reformulada)

### ChatPromptTemplate

In [ ]:
def answer_template(language="spanish"):
    template = f"""Eres un asistente especializado en la Constitución Política del Estado Plurinacional de Bolivia.
Responde la siguiente pregunta utilizando únicamente la información proporcionada en el contexto (delimitado por <context>).
Tu respuesta debe estar en el idioma del final.

Si la información no es suficiente o no se encuentra en el contexto, responde claramente:
"No tengo información suficiente en la Constitución para responder a esta pregunta."

<context>
{{context}}
</context>

Pregunta: {{question}}

Idioma: {language}.
"""
    return template

In [ ]:
def build_answer_prompt(memory, context, question, language="Spanish"):
    chat_template = answer_template(language=language)
    chat_history = memory.get_context() if memory else "No hay historial previo."

    prompt = chat_template.format(
        context=context,
        chat_history=chat_history,
        question=question
    )
    return prompt

In [ ]:
memory = ConversationMemory(max_turns=5)
memory.add_message("user", "¿Qué dice el artículo 7 de la Constitución?")
memory.add_message("assistant", "El Artículo 7 establece los principios fundamentales del Estado boliviano.")

pregunta_final = "¿Qué sucede si no se respetan los derechos fundamentales establecidos en el artículo 7?"

resultados = retriever_gemini.invoke(pregunta_final)
contexto = "\n".join(resultados["documents"][0])
prompt_final = build_answer_prompt(memory, contexto, pregunta_final, language="Spanish")
print(prompt_final)

In [ ]:
respuesta = llm.invoke(prompt_final)
print("\n🔹 RESPUESTA DEL MODELO 🔹")
print(respuesta)